## 第4章 基于概率论的分类方法：朴素贝叶斯

### 本章内容

- 使用概率分布进行分类
- 学习朴素贝叶斯分类器
- 解析RSS源数据
- 使用朴素贝叶斯来分析不同地区的态度



前两章我们要求分类器给出“”该数据实例属于哪一类“这类问题的明确答案。不过分类器有时会产生错误结果。这是可以要求分类器给出一个最优的类别猜测结果，同时给出这个猜测的概率估计值。

概率论使许多机器学习算法的基础。所以深刻理解这一主题就显得十分重要。在第3章计算特征值取某个值的概率时涉及了一些概率知识，在那里我们先统计特征在数据集中取某个特定值的次数，然后除以数据集的实例总数，就得到了特征取该值的概率。我们将在此基础上深入讨论。

本章会给出一些使用概率论进行分类的方法。首先从最简单的概率分类器开始，然后给出一些假设来学习朴素贝叶斯分类器。我们称之为”朴素“，是因为整个形式化过程只做最原始、最简单的假设。不必担心，你会详细了解到这些假设。我们将充分利用Python的文本处理能力将文档分切成词向量，然后利用词向量对文档进行分类。我们还将构建另一个分类器，观察其在真实的垃圾邮件数据集中的过滤效果，必要时还会回顾一下条件概率。最后，我们将介绍如何从个人发布的大量广告中学习分类器，并将学习结果专程人类可理解的信息。

### 4.1 基于贝叶斯决策理论的分类方法

朴素贝叶斯

优点：在数据较少的情况下仍然有效，可以处理多类别问题。

缺点：对于输入数据的准备方式较为敏感

适用数据类型：标称型数据

朴素贝叶斯是贝叶斯决策理论的一部分，所以讲述朴素贝叶斯之前有必要快速了解一下贝叶斯决策理论。假设现在我们有一个数据集，它由两类数据组成，数据分布如图4-1所示。


<img src="./4-1.jpg" alt="4-1" style="zoom:33%;" />


图4-1 两个参数已经的概率分布，参数决定了分布的形状



假设有位读者找到了描述途中两类数据的统计参数。（暂且不用管如何找到描述这类数据的统计参数，第10章会详细介绍。）我们现在用p1（x,y）表示数据点（x,y）属于类别1（图中圆点表示的类别）的概率，用p2（x,y）表示数据点（x,y）属于类别2（图中三角形表示的类别）的概率，那么对于一个新数据点（x,y）,可以用下面的规则来判断它的类别：

- 如果p1（x,y）> p2(x,y)那么类别为1.
- 如果p2（x,y）> p1(x,y) 那么类别为2.

也就是说，我们会选择高概率对应的类别。这就是贝叶斯决策理论的核心思想。即选择具有最高概率的决策。回到图4-1，如果该图中的整个数据使用6个浮点数来表示，并且计算类别概率的Python代码只有两行，那么你会更倾向于使用下面哪种方法来对该数据点进行分类？

（1）使用第1章的KNN，进行1000次距离计算

（2）使用第2章的决策树，分别沿x轴、y轴划分数据；

（3）计算数据点属于每个类别的概率，并进行比较。

使用决策树不会非常成功；而和简单的概率计算相比，kNN的计算量太大，因此，对于上述畏难而退，最佳选择是使用刚才提到的概率比较方法。

接下来，我们必须要详述p1及p1概率计算方法。为了能够计算p1与p2，有必要讨论一下条件概率。如果你觉得自己已经相当了解条件概率了，那么可以世界跳过下一节。



贝叶斯？

这里使用的概率解释属于贝叶斯概率理论的范畴，该理论非常流行且效果良好。贝叶斯概率以18世纪的一位神学家托马斯贝叶斯（Thomas Bayes）的名字命名的。贝叶斯概率引入先验知识和逻辑推理来处理不明确命题。另一种概率解释称为频数概率（frequency probability），它只从数据本身获得结论，并不考虑逻辑推理及先验知识。

### 4.2 条件概率

接下来花点时间将将概率与条件概率。如果你对p（x,ylc）符号很熟悉，那么可以跳过本节。假设现在有一个装了7块石头的罐子，其中3块是灰色的，4块是黑色的（如图4-2），如果从管子中随机抽取一块石头，那么灰色石头的概率是3/7，黑色石头概率4/7。

<img src="./12901600832632_.pic.jpg" alt="12901600832632_.pic" style="zoom:33%;" />


如果这7块石头在两个桶中，上述概率如何计算？



<img src="./12911600832977_.pic.jpg" alt="12911600832977_.pic" style="zoom:33%;" />

要计算P（gray）或者P（black），事先得知道石头所在桶的信息会不会改变结果？你有可能已经想到计算从B桶中取到灰色石头的概率的办法，这就是所谓的条件概率（conditional probability）。假定计算的是从B桶取到灰色石头的概率，这个概率可以记作P（gray|bucketB）,我们称之为“在已知石头出自B桶的条件下，取出灰色石头的概率”。不难得到，P（gray|bucketA） 值为2/4，P（gray|bucketB）的值为1/3。

条件概率的计算公式如下所示：
<img src="./12921600833413_.pic.jpg" alt="12921600833413_.pic.jpg" style="zoom:66%;" />



首先用B桶中灰色石头的个数除以两个桶中总的石头树，得到P（gray and bucketB） = 1/7。其次，由于B桶中有3块石头，而总石头数为7，于是P（bucketB）就等于3/7 。于是有P（gray|bucketB）= P（gray and bucketB）/P（bucketB）=（1/7）/（3/7） = 1/3。这个公式虽然对于这个简单的例子来说有点复杂，但当粗在更多特征时时非常有效的。用代数方法计算条件概率是，该公式也很有用。

另一种有效计算条件概率的方法称为贝叶斯准则。贝叶斯准则告诉我们如何交换条件概率中的条件与结果，即如果已知P（x|c），要求P（c|x），那么可以使用下面的计算方法：
<img src="./12931600834750_.pic.jpg" alt="/12931600834750_.pic.jpg" style="zoom:66%;" />



我们讨论了条件概率，接下来的问题是如何将其应用到分类器中。下一节将套六年如何结合贝叶斯决策理论使用条件概率。条件概率是指事件A在另外一个事件B已经发生条件下的发生概率。条件概率表示为：P（A|B），读作在“B条件下A的概率”条件概率可以用作决策树进行计算。



### 4.3 使用条件概率来分类

4.1节体罚哦的贝叶斯决策理论要求计算机两个概率p1（x，y）和p2（x, y）:

- 如果p1(x,y)>p2(x, y) ,那么属于类别1；
- 如果p2(x,y)>p1(x,y)，那么属于类别2。

但这两个准则并不是贝叶斯决策理论的所有内容。使用p1（）和p2（）只是为了尽可能简化描述，而真正需要计算和比较的是 $$p(c{_1}|x,y)$$ 和 $$p(c{_2}|x,y)$$ ，这些符号所代表的具体意义是：给定某个由x、y表示的数据点，那么该数据点来自类别 $ c_1$ 的概率是多少？数据点来自类别$c_2$ 的概率又是多少？注意这些概率与刚才给出的概率$p(x, y|c_1)$ 并不一样，不过可以使用贝叶斯准则来交换概率中条件与结果。具体地，应用贝叶斯准则得到：

$p(c_i|x,y) = \frac{p(x,y|c_i)p(c_i)}{p(x,y)}$

使用这些定义，可以定义贝叶斯分类准则为：

- 如果$P(c_1|x,y) >P(c_2|x,y)$,那么属于类别$c_1$;
- 如果$P(c_1|x,y)$<$P(c_2|x,y)$,那么属于类别$c_2$;

使用贝叶斯准则，可以通过已知的三个概率值来计算为止的概率值。后面就会给出利用贝叶斯准则来计算概率并对数据进行分类的代码。。现在介绍了一些概率理论。你也了解了基于这些理论构建分类器的方法，接下来就要将它们付诸实践。下一节会介绍一个简单但功能强大的贝叶斯分类器的应用案例。



### 4.4 使用朴素贝叶斯进行文档分类

机器学习的一个重要应用就是文档的自动动分类，在文档分类中，整个文档（如一封电子邮件）是实例，而电子邮件中的某些元素则构成特征虽然电子邮件是一种会不断增加的文本，但是我们同样也可以对新闻报道、用户留言、政府公文等其他任意类型的文本进行分类。我们可以观察文档中出现的词，并把每个词的出现或者不出现作为一个特征，这样得到的特征数目就会跟词汇表中的词目一样多。朴素贝叶斯是上节介绍的特叶斯分剋起的一个扩展，是用于文档分类的常用算法。

使用每个词作为特征并观察它们是否出现，这样得到的特征数目会有多少呢？针对的是哪一种人类语言呢？当然不止一种语言。据估计，仅仅 在英语中，单词的总数就有500000之多，为了能进行英文阅读，估计需要掌握数千单词。



朴素贝叶斯的一般过程

（1）收集数据：可以使用任何方法。本章使用RSS源。

（2）准备数据：需要数值型或者布尔型数据。

（3）分析数据：有大量特征时，绘制特征作用不大，此时使用直方图效果更好。

（4）训练算法：计算不同的独立特征的条件概率

（5）测试算法：测试错误率

（6）使用算法：一个常见的朴素贝叶斯应用时文档分类。可以在任意的分类场景中使用朴素贝叶斯分类器，不一定非要是文本。



假设词汇表中有1000个单词，要的好的概率分布，就需要有足够的数据样本，假定样本数为N。前面讲到的约会网站示例中有1000个实例，手写识别示例中每个数字由200个样本，而决策树示例中有24个样本。其中，24个样本有点少，200个样本好一些，而1000个样本就非常好了。约会网站例子中有三个特征。由统计学知，如果每个特征需要N个样本，那么对于10个特征将需要$N^{10}$,包含1000个特征的词汇表需要 $N^{1000}$个样本。可以看到，所需要的样本数会随着特征数目增大而迅速增长。

如果特征之间相互独立，那么样本数就可以从$N^{1000}$ 减少到1000$*$N 。所谓独立Independent指的是统计意义上的独立，即一个特征或者单词出现的可能性与它和其他单词相邻没有关系。举个例子，假设单词bacon出现在unhealthy后面与出现在delicious 后面的概率相同。当然，我们知道这种假设并不正确，bacon常常出现在delicious 附近，而很少出现在unhealthy附近，这个假设正是朴实贝叶斯分类器中朴素（naive）一词的含义。朴素贝叶斯分类器中的另一个假设是，每个特征同等重要。其实这个假设也有问题。如果要判断留言板的留言是否得当，那么可能不需要看完所有的1000个单词，而只需要看10～20个特征就足以做出判断了。尽管上述假设存在一些小的瑕疵，但朴素贝叶斯的实际效果却很好。

到目前为止，你已经了解了足够的知识，可以开始编写代码了。下一节将使用Python来实现朴素贝叶斯分类器，实现中会涉及利用Python来进行文本分类的所有相关内容。

### 4.5 使用Python进行文本分类

从文本中获取特征，需要拆分文本，具体如何做呢？这里的特征是来自文本的词条（token），一个字条时字符的任意组合。可以把词条想象为单词，也可以使用费单词词条。如URL、IP地址或者任意其他字符串。然后将每一个文本片段表示为一个词条向量，其中值为1 的表示出现子文档中，0表示未出现。

过滤留言板的侮辱性语言，我们要构建一个快速过滤器。如果某条留言使用了负面或者侮辱性语言，那么就将该留言标识为内容不当。过滤这类内容是一个很常见的需求。对此问题建立两个类别：侮辱类和非侮辱类，使用1和0分表表示。

接下来首先给出将文本转换为数字向量的过程，然后介绍如何给予这些向量来计算条件概率，并在此基础上构建分类器，卒子后还要介绍一些利用Python实现朴素贝叶斯过程中需要考虑的问题。



#### 4.5.1 准备数据：从文本中构建词向量

我们将把文本看成单词向量或者词条向量，也就是说将橘子转换为向量。考虑出现在所有文档中的所有单词，再决定将那些词纳入词汇表或者说所要的词汇集合，然后必须要将每一篇文档转换为词汇表上的向量。

$$程序清单4-1词表到向量的转换函数$$


In [20]:
from numpy import *

In [21]:
def loadDataSet():
    postingList = [['my','dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless','garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying','worthless','dog','food','stupid']]
   

    #     1代表侮辱性文字，0代表正常言论
    classVec = [0,1,0,1,0,1]   
    return postingList,classVec

def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet|set(document)
    return list(vocabSet)

def setOfWords2Vec(vocabList,inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:print("the word: %s is not in my Vocabulary!"%word)
    return returnVec

    

函数loadDataSet() 创建了一些实验样本。该函数返回的第一个变量是进行词条切分后的文档集合，这些文档来自斑点犬爱好者留言板。这些留言文本被切分成一系列的词条集合，标点符号从文本中去掉，后面或探讨文本处理的细节。loadDataSet() 函数返回的第二个变量是一个类别表签的集合。这里有两类，侮辱性和非侮辱性。这些文本的类别由人工标注，这些标注信息用于训练程序以便自动检测侮辱性留言。

下一个函数createVocabList()会创建一个包含在所有文档中出现的不重复词的列表。为此使用了Python的Set数据类型。将词条列表输给Set构造函数，set就会返回一个不重复词表。首先，创建一个空集合，然后将每篇文档返回的新词集合添加到该集合中。操作符|用于求两个集合的并集。这也是一个按位或（OR）操作符（参见附录C）。在数学符号表示上，按位或操作与集合求并操作使用相同的记号。

获得词汇表后，便可以使用函数setOfWord2Vec(),该函数的输入参数为词汇表及某个文档，输出的是文档向量，向量的每一个元素都为1或0，分别表示词汇表中的残次在输入文档中是否出现。函数首先创建一个和词汇表等长的向量，并将其元素都设置为0.接着遍历文档中所有的单词，如果出现了词汇表中的单词，则将输出的文档向量中的对应值设为1。一切都顺利的话，就不需要检查某个词是否还在vocabList中，后边可能会用到这一操作。 现在看一下函数执行效果：


In [22]:
listOPosts,listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)

In [23]:
myVocabList


['help',
 'ate',
 'mr',
 'steak',
 'how',
 'stop',
 'licks',
 'worthless',
 'quit',
 'problems',
 'him',
 'has',
 'park',
 'love',
 'dog',
 'garbage',
 'dalmation',
 'so',
 'stupid',
 'to',
 'take',
 'not',
 'is',
 'I',
 'buying',
 'my',
 'food',
 'please',
 'flea',
 'cute',
 'posting',
 'maybe']

检查上述词表，就会发现这里不会出现重复的单词。目前该词表还没有排序，需要的话，稍后可以排序。下面看一下函数setOfWords2Vec()的运行效果：


In [24]:
setOfWords2Vec(myVocabList,listOPosts[0])

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0]

In [25]:
listOPosts[0]


['my', 'dog', 'has', 'flea', 'problems', 'help', 'please']

In [26]:
listOPosts

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

In [27]:
setOfWords2Vec(myVocabList,listOPosts[3])

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

该函数书用词汇表喙着想要检查的所有单词作为输入，然后为其中某一个单词构建一个特征。一旦给定一篇文档（斑点犬的一条留言），该文芳就会被转换为词向量。接下来检查一下函数的有效性。myVocabList中出现了单词help，我们检查一下它是否出现在第四篇文档中。

该函数书用词汇表喙着想要检查的所有单词作为输入，然后为其中某一个单词构建一个特征。一旦给定一篇文档（斑点犬的一条留言），该文芳就会被转换为词向量。接下来检查一下函数的有效性。myVocabList中出现了单词help，我们检查一下它是否出现在第四篇文档中。



#### 4.5.2 训练算法：从词向量计算概率

前面介绍了如何将一组单词转换为一组数字，接下来看看如何适应这些数字计算概率。现在已经知道一个词是否出现在一篇文档中，也知道该文档所属的类别。还记得4.2节一道的贝叶斯准则？我们重写贝叶斯准则，将之前的x、y替换成**W**。**W**表示这是一个向量，即它由多个数值组成。在这个例子中，数值个数与词汇表中的词个数相同。

$p(c_i|w = \frac{p(w|c_i)p(c_i)}{p(c_i|w)})$

我们将使用上述公式，对每个类计算该值，然后逼叫这两个概率值的大小。如何计算呢？首先可以通过类别i（侮辱性留言或非侮辱性留言）中文档数除以总的文档数来计算概率$p(c_i$),接下来计算$p(w|c_i)$,这里就要用到朴素贝叶斯假设。如果将$w$展开为一个个独立特征，那么就可以将上述概率写作$p(w_0,w_1,w_2..w_n|c_i)$这里假设所有词都互相独立，该假设也称作条件独立性假设，它意味着可以使用$p(w_0|c_i)p(w_1|c_i)p(w_2|c_i)..p(w_n|c_i)$来计算上述概率，这就极大地简化了计算的过程。




该函数的伪代码如下：


计算每个类别中的文档数目对每篇训练文档：

    对每个类别：

		如果词条出现在文档中—>增加该词条的计数值

		增加所有词条的计数值对每个类别；

    对每个词条：

		将该词条的数目除以总词条数目得到条件概率返回每个类别的条件概率




我们利用下面的代码来实现上述伪代码：


$$程序清单4-2 朴素贝叶斯分类器训练函数$$

In [28]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = zeros(numWords);p1Num = zeros(numWords)
    p0Denom = 0.0;p1Denom = 0.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num+=trainMatrix[i]
            p1Denom +=sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = p1Num/p1Denom  #change to log()
    p0Vect = p0Num/p0Denom #change to log（）
    return p0Vect,p1Vect,pAbusive




代码函数中的输入参数为文档矩阵trainMatrix，以及有每篇文档类别标签所构成的向量trainCategory。首先，计算文档属于侮辱性文档（class = 1）的概率，即P(1)。因为这是一个二类分类问题，所以可以通过1-P(1) 得到P (0)。对于多于两类的分类问题，则需要对代码稍加修改。

计算$p(w_i|c_i)$ 和$p(w_i|c_0)$, 需要初始化程序中的分子和分母变量。由于$w$中元素如此众，因此可以使用numpy数组快速计算这些值。上述程序中的分母变量是一个元素等于词汇表大小的numpy数组。在for循环中，要遍历训练集trainMatrix中的所有文档。一旦某个词语（侮辱性或正常词语）在某一文档中出现，该词对应的个数（p1Num或者p0Num）就加1，而起在所有的文档中，该文档的总词数也相应加1，对于两个类别都要进行同样的计算处理。

最后对每个元素除以该类别中的总词数。利用Numpy可以很好实现，用一个数组除以浮点数即可。若使用常规的python列表则难以完成这种任务。最后，函数会返回一个向量和两个概率。接下来实验一下：

In [29]:
listOPosts,listClasses = loadDataSet()




该语句从预先加载值中调入数据。

In [30]:
myVocabList = createVocabList(listOPosts)

至此我们构建了一个包含所有词的列表myVocabList

In [31]:
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))

该for循环使用词向量来填充trainMat列表。下面给出属于侮辱性文档的概率以及两个类别的概率向量。

In [32]:
p0V,p1V,pAb = trainNB0(trainMat,listClasses)

接下来看这些变量的内部值：

In [33]:
pAb

0.5

这就是任意文档属于侮辱性文档的概率。



In [34]:
p0V

array([0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
       0.04166667, 0.04166667, 0.        , 0.        , 0.04166667,
       0.08333333, 0.04166667, 0.        , 0.04166667, 0.04166667,
       0.        , 0.04166667, 0.04166667, 0.        , 0.04166667,
       0.        , 0.        , 0.04166667, 0.04166667, 0.        ,
       0.125     , 0.        , 0.04166667, 0.04166667, 0.04166667,
       0.        , 0.        ])

In [35]:
len(p0V)

32

In [36]:
p1V

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.05263158, 0.        , 0.10526316, 0.05263158, 0.        ,
       0.05263158, 0.        , 0.05263158, 0.        , 0.10526316,
       0.05263158, 0.        , 0.        , 0.15789474, 0.05263158,
       0.05263158, 0.05263158, 0.        , 0.        , 0.05263158,
       0.        , 0.05263158, 0.        , 0.        , 0.        ,
       0.05263158, 0.05263158])

首先我们发现文档属于侮辱类的概率pAb为0.5.该值是正确的。接下来，看一看在给定文档类别条件下词汇表中单词的出现概率，看看是否正确。词汇表中的第一个词是cute，其在类别0中出现1次，而在类别1只能够从未出现。对应的条件概率分别为0.4166667与0.0。该计算是正确的。我们找找所有概率中的最大值。该值出现在P(1)数组的0.1578947是最能表征类别（侮辱性文档类）的单词。使用该函数进行分类前，还需要解决函数中的一些缺陷。

#### 4.5.3 测试算法：根据显示情况修改分类器

利用贝叶斯分离器对文档进行分类事，要计算多个概率的乘积以获得文档属于某个类别的概率，即计算$p(w_0|1)p(w_1|1)p(w_2|1)$ 。如果其中图个概率值为0，那么最后的乘积也为0.为降低这种影响，可以将所有的词出现初始化为1，并将分母初始化为2。


####  p0Num = ones(numWords);p1Num = ones(numWords)
####  p0Denom = 2.0;p1Denom = 2.0|

另一个遇到的问题是下溢出。这是由于套多很小的数相乘造成的。当计算乘积$p(w_0|c_1)p(w_1|c_i)p(w_2|c_i)...p(w_N|c_i)$时，由于大部分因子都给畅销，所以程序会下溢出或者得到不正确的答案。（读者可以用Python尝试相乘许多很小的树，最后四舍五入后会得到0.）一种解决帮时对于乘积取自然对数。在代数中有$ln(a*b)=ln(a)+ln(b)$, 于是通过求对树可以避免下溢出或者浮点数舍入导致的错误。同时，采用自然对数进行处理不会有任何损失。图4-4给出函数f(x)与ln(f(x))的曲线。检查这两条曲线，就会发现它们在相同区域内同时增加或者减少，并且在相同电商取到极值。它们的取值虽然不同，但不影响最终结果。通过修改return前的两行代码，将上述做法用刀分类器中：
<img src="./image-20200924075928815.png" alt="image-20200924075928815" style="zoom:33%;" />

$$函数f(x) 与ln(f(x))会一块增大。这表明相求函数的最大值时，可以使用该函数的自然对数来替换原函数进行求解。$$




 #### p1Vect = log(p1Num/p1Denom)  #change to log()
 #### p0Vect = log(p0Num/p0Denom) #change to log（）

In [37]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = ones(numWords);p1Num = ones(numWords)
    p0Denom = 2.0;p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num+=trainMatrix[i]
            p1Denom +=sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom)  #change to log()
    p0Vect = log(p0Num/p0Denom) #change to log（）
    return p0Vect,p1Vect,pAbusive

现在已经准备好构建完整的分类器了。当使用numpy向量处理功能时，这一切变得十分简单。

$$程序清单4-3朴素贝叶斯分类函数$$

In [38]:

def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1 = sum(vec2Classify*p1Vec)+log(pClass1)
    p0 = sum(vec2Classify*p0Vec)+log(1.0-pClass1)
    
    if p1>p0:
        return 1
    else:
        return 0
    
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love','my','dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid','garbage']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))

程序清单4-3 的代码有4个输入：要分类的向量vec2Classify以及使用函数trainNB0()计算得到的三个概率。使用Numpy的数组来计算两个向量相乘的结果。这里的相乘时对应元素相乘，即先将两个向量中的第1个元素相乘，然后将第2个元素相乘，以此类推。接下来将词汇表中的所有词的对应值相加，然后将该值加到类别的对数概率上。最后，比较类别的概率返回大概率对应的类别标签。这一切不是很难，对吧？

代码的第二个函数是一个便利函数（convenience function）,该函数封装所有操作，以节省输入4.3.1节中代码的时间。下面来看看实际结果。


In [39]:
testingNB()

['love', 'my', 'dalmation'] classified as: 0
['stupid', 'garbage'] classified as: 1


对文本做一些修改看看分类器会输出什么结果。这个例子非常简单但是它展示了朴素贝叶斯分类器的工作原理。接下来，我们会对代码做些修改，使分类器工作得更好。

#### 4.5.4 准备数据：文档词袋模型

目前为止，我们将每个词的出现与否作为一个特征，这可以被描述为词集模型（set-of-words model）。如果一个词在文档中出现不止一次，这可能意味着包含该词是否出现在文档中所不能表达的某种信息，这种方法被称为词袋模型（bag-of-words model）。在词袋中，每个单词可以出现多次，而在词集中，每个词只能出现一次。为适应词袋模型，需要对函数setOfWords2Vec()稍加修改，修改后的函数称为bagOfWords2Vec()。

下面的程序清单给出了基于词袋模型的朴素贝叶斯代码。它与函数setOfWords2Vec()几乎完全相同，唯一不同的是每当遇到一个单词时，它会增加词向量中的对应值，而不是将对应的数值设为1.

$$程序清单4-4 朴素贝叶斯词袋模型$$



In [40]:
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]+=1
    return returnVec

In [41]:
现在分类器已经构建好了，下面我们将利用该分类器来过滤垃圾邮件。

SyntaxError: invalid character in identifier (<ipython-input-41-d05c670e9699>, line 1)

### 4.6 示例：使用朴素贝叶斯过滤垃圾邮件

在前面那个简单的例子中，我们引入了字符串列表。使用朴素贝叶斯解决一些显示生活中的问题时，需要先从文本内容得到字符串，然后生成词向量。下面这个例子中，我们将了解朴素贝叶斯的一个最著名的应用：电子邮件垃圾过滤。首先看一下如何使用通用框架来解决该问题：





示例：使用朴素贝叶斯队电子邮件进行分类

（1）收集数据：提供文本文件

（2）准备数据：将文本文件解析成词条向量

（3）分析数据：检查词条确保解析的正确性

（4）训练算法：使用我们之前建立的trainNB0（）

（5）测试算法：使用classifyNB（）,并且构建一个新的测试函数来计算文档集的错误率

（6）使用算法：构建一个完整的程序对一组文档进行分类，将错分的文档输出到屏幕上。



下面首先给出将文档解析为词条的代码。然后将该代码和前面的分类代码集称为一个函数，该函数在测试分类器的同时会给出错误率。

#### 4.6.1 准备数据：切分文本

前一节介绍了如何创建词向量，并基于这些词向量进行朴素贝叶斯分类的过程。前一节中的词向量是预先给定的，下面介绍如何从文本文档中购进自己的词列表。

对于一个文本字符串，可以使用python的string.split()方法将其切分。下面看看实际的运行效果。





In [42]:
mySent = 'This book is the best book on Python or M.L. I have ever laid eyes upon.'
mySent.split()


['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M.L.',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon.']

可以看到，切分的结果不错，但是标点符号也被当成了词的一部分。可以使用正则表示来切分句子，其中分隔符是除单词、数字外的任意字符串。

In [43]:
import re
regEx = re.compile('\\W*')
listOfTokens = regEx.split(mySent)
listOfTokens


/Users/TangoTian/Venvs/algo/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: split() requires a non-empty pattern match.
  This is separate from the ipykernel package so we can avoid doing imports until


['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M',
 'L',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon',
 '']

现在得到了一系列词组成的词表，但是里面的空字符串需要去掉。可以计算每个字符串的长度，只返回长度大于0的字符串。

In [44]:
[tok for tok in listOfTokens if len(tok)>0]

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M',
 'L',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

最后，我们发现句子中的第一个单词时大写的。如果是橘子查找，那么这个特点喙很有用。但是这里的文本只看成词袋，所以我们希望所有词的形式都是统一的，不论它们出现在句子中间、结尾还是开头。

Python中有一些内嵌的方法，可以将字符串全部转换成小写 (.lower())或者大写（.upper(),借助这些方法可以达到目的。于是，可以进行如下处理：

In [45]:
[tok.lower() for tok in listOfTokens if len(tok)>0]

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'm',
 'l',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

现在看来数据集中一封完整的电子邮件的实际处理结果。该数据集放在email文件夹中，该文件夹又包含连个子文件夹，分别是spam与ham。

In [46]:
emailText = open('email/ham/6.txt',encoding='utf-8',errors='ignore').read()
listOfToken=regEx.split(emailText)

/Users/TangoTian/Venvs/algo/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: split() requires a non-empty pattern match.
  


文件夹ham下的6.txt文件非常长，这是某公司告知他们不再进行某些支持的一封邮件。需要注意的是，由于是url:answer.py?hl=en&amswer = 174623的一部分，因而会出现en和py这样的单词。当对URL进行切分时，会得到很多的词。我们是想去掉这些单词，因此在实现时会过滤掉长度小于3的字符串。本例使用一个通用的文本解析规则来实现这一点。在世纪的解析程序中，要用更高级的过滤器来对诸如HTML和URL的对象进行处理。目前一个url最终会解析成词汇表中的单词，比如www.whitehouse.gov  会被解析为三个单词。文本解析可能是一个相当复杂的过程。接下来将构建一个机器简单的函数，你可以根据情况自行修改。

#### 4.6.2 测试算法：使用朴素贝叶斯进行交叉验证

下面将文本解析器即成到一个完整的分类器中。

$$程序清单4-5 文件解析及完整的垃圾邮件测试函数$$

In [47]:
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W*',bigString)
    return [tok.lower() for tok in listOfTokens if len(tok)>2]

def spamTest():
    docList = []; classList = []; fullText = []
    
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt'%i,encoding='utf-8',errors='ignore').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt'%i,encoding='utf-8',errors='ignore').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    trainingSet = list(range(50));testSet = []
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
#         print("-----randIndex",randIndex,"------trainingSet",trainingSet)
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat = []; trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList,docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam)!=classList[docIndex]:
            errorCount += 1
    print('the error rate is ',float(errorCount)/len(testSet))
        
        
        
        
        

第一个函数textParse() 接受一个大的字符串并将其解析为字符串列表。该函数取消少于两个字符的字符串并将其解析为字符串列表。该函数去掉少于两个字符的字符串，并将所有的字符串转换为小写。你可以在函数中添加更多的解析操作。但是目前的实现对于我们的应用足够了。

第二个函数spanTest（）对贝叶斯垃圾邮件分诶器进行自动化处理。倒入文件spam与ham下的文本文件。并将它们解析为词列表。接下来构建一个测试集于一个训练集。两个集合的邮件都是随机选出的。本例中共有50封帝啊你邮件，并不是很多，其中的10封电子邮件被随机选择为测试集。分类器所需要的概率计算只利用训练集中的文档来完成。Python变量trainingSet是一个整数列表，其中的值从0到49.接下来，随机选其中10个文件。选择出的数字所对应的文档被添加到测试集。同时也将其从训练集中剔除。这种随机选择数据的一部分作为训练集，而剩余部分作为测试集的过程称为留存交叉验证（hold-out cross validation）。假定现在只完成了一次迭代，那么为了更精确地估计分类器的错误率，就应进行多次迭代后求出平均错误率。

接下来的for循环遍历训练集所有文档，对，饿哦风邮件基于词汇表并使用setOfWord2Vec()函数来构建词向量。这些词在trainNB0()函数中用于计算分类所需的概率。然后遍历测试集，最其中美风电子邮件进行分类。如果邮件分类错误，则错误加1，

最后给出的总错误百分比。下面对上述过程进行 尝试。




In [48]:
spamTest()

the error rate is  0.2


/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [49]:
spamTest()

the error rate is  0.1


In [50]:
spamTest()

the error rate is  0.0


In [51]:
spamTest()

the error rate is  0.0


In [52]:
spamTest()

the error rate is  0.0


In [53]:
spamTest()

the error rate is  0.1


In [54]:
spamTest()

the error rate is  0.0


In [55]:
spamTest()


the error rate is  0.0


In [56]:
spamTest()

the error rate is  0.1


In [57]:
spamTest()

the error rate is  0.0


In [58]:
spamTest()

the error rate is  0.1


In [59]:
spamTest()

the error rate is  0.0


In [60]:
spamTest()

the error rate is  0.1


In [61]:
spamTest()

the error rate is  0.1


In [62]:
spamTest()

the error rate is  0.0


In [63]:
spamTest()

the error rate is  0.0


In [64]:
spamTest()

the error rate is  0.0


In [65]:
spamTest()


the error rate is  0.2


In [66]:
spamTest()

the error rate is  0.0


In [67]:
spamTest()

the error rate is  0.0


In [68]:
spamTest()


the error rate is  0.0


In [69]:
spamTest()

the error rate is  0.0


In [70]:
spamTest()

the error rate is  0.0


In [71]:
spamTest()

the error rate is  0.0


In [72]:
spamTest()

the error rate is  0.0


In [73]:
spamTest()

the error rate is  0.1


In [74]:
spamTest()

the error rate is  0.1


In [75]:
spamTest()

the error rate is  0.1


In [76]:
spamTest()


the error rate is  0.0


函数spamTest ()会输出在10封随机选择的电子邮件上的分类错误率。既然这些电子邮件是随机选择的，所以每次输出的结果可能有些差别。如果发现错误的话，函数会输出错分文档的词汇表。这样就可以了解到底是哪篇文档发生了错误。如果想要更好地估计错误率，那么就应该将上述过程重复多次，比如说10次，然后求平均值。为避免错误，有多种方式可以用来修正分类器，这些将在第 7章中进行讨论。

目前我们已经使用朴素贝叶斯来对文档进行分类，接下来将介绍它的另外一个应用。下一个例子还会展示如何解释朴素贝叶斯分类器训练所得到的知识


### 4.7 示例：使用朴素贝叶斯分类器从个人广告中获取区域倾向

本章的最后一个例子非常有趣，我们前面介绍了朴素贝叶斯的两个实际应用的立足，第一个例子是过滤网站的恶意留言，第二个是过滤垃圾邮件。分类还有大量的其他应用。我曾见过有人使用朴素贝叶斯从他喜欢及不喜欢的女性的社交网络档案中学习相应的分类器，然后利用该分类器测试他是否会喜欢上一个陌生女人。分类的可能应用确实有很多，比如有证据表示，人的年龄越大，他所用的词也就越好。那么，可以基于一个人的用词来推测他的年龄吗？除了年龄之外，还能否推测其他方面？广告商往往想知道关于一个热的一些特定人口统计信息。以便能够更好地定向推销广告。从哪里可以获得这些训练数据呢？事实上，互联网上拥有大量的训练数据。几乎人一个能想到的利基市场都有专业社区，很多人会认为自己属于该社区。4.5.1节中的斑点犬爱好者网站就是一个非常好的例子。

在这个最后的瘤子当中，我们将分别从美国的两个城市中选取一些人。通过分析着喜人发布的征婚广告信息，来比较这两个城市的人们在广告用词上是否不同。如果结论确实是不同，那么他们各自常用词是哪些？从人民的用词当中，我么能否对不同城市的人所关心的内容有所了解？



示例：使用朴素贝叶斯来发现地域相关的用词

（1）收集数据：从RSS源收集内容，这里需要对RSS源狗卷一个接口。

（2）准备数据：降温测吧文件解析成词条向量

（3）分析数据：检查词条确保解析的正确性

（4）训练算法：使用我们之前建立的trainNB0（）函数；

（5）测试算法：观察错误率，确保分类器可以用。可以修改切分程序，以降低错误率，提高分类结果。

（6）使用算法：构建一个完整的程序，分装所有内容。给定两个RSS源，该程序会显示最常用的公共词。



下面将使用来自不同城市的广告训练一个分类器，然后观察分类器的效果。我们的目的并不是使用该分类器进行分类，而是通过观察单词和条件概率值来发现与特定城市相关的内容。

#### 4.7.1 收集数据：蹈入RSS源



接下来要做的第一件事是使用Python下载文本。幸好，利用RSS，这些文本很容易得到。现在所需要的是一个RSS阅读器。Universal feed Parser是Python中最常用的RSS程序库。

你可以在http://code.google.com/p/feedparser/下浏览文档，然后和其他Python包一样来安装feedparse。首先解压下载的包。并将当前没有了 u切换到解压文件所在的文件夹。然后在python提示符下敲入 python setup.py install .

由于原作使用的Craigslist上的RSS源地址已经失效，大家可以找有效的RSS代替。（可参考 【RSS 订阅精选 2020】http://www.bubuko.com/infodetail-3446798.html）， 这里我使用了《界面新闻》的RSS 地址 https://a.jiemian.com/index.php?m=article&a=rss



In [77]:
 
import feedparser
ny = feedparser.parse('https://a.jiemian.com/index.php?m=article&a=rss')

In [78]:
ny


{'bozo': False,
 'entries': [{'title': '旗引天山南北 谱写发展新篇——以习近平同志为核心的党中央关心新疆工作纪实',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://a.jiemian.com/index.php?m=article&a=rss',
    'value': '旗引天山南北 谱写发展新篇——以习近平同志为核心的党中央关心新疆工作纪实'},
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'https://www.jiemian.com/article/5034981.html'}],
   'link': 'https://www.jiemian.com/article/5034981.html',
   'summary': '<img src="https://img1.jiemian.com/101/original/20200924/160094832544655800_a580x330.jpg" /><p>祖国的西北角，是166万多平方公里的大美新疆。生活在这片土地上的56个民族、2500万中华儿女守望相助、不懈奋斗，共同建设美丽家园。</p>',
   'summary_detail': {'type': 'text/html',
    'language': None,
    'base': 'https://a.jiemian.com/index.php?m=article&a=rss',
    'value': '<img src="https://img1.jiemian.com/101/original/20200924/160094832544655800_a580x330.jpg" /><p>祖国的西北角，是166万多平方公里的大美新疆。生活在这片土地上的56个民族、2500万中华儿女守望相助、不懈奋斗，共同建设美丽家园。</p>'},
   'published': 'Thu, 24 Sep 2020 19:57:38 +0800',
   'published

In [79]:
sf = feedparser.parse('http://news.stockstar.com/rss/xml.aspx?file=xml/stock/2.xml')

In [80]:
len(sf['entries'])

30

In [81]:
len(ny['entries'])

30

可以构建一个类似于spamTest（）所谓函数来对测试过程自动化。打开文本编辑器，输入下列程序清单中的代码。

$$程序清单4-6 RSS 源分类器及高频词去除函数$$

In [87]:
def calcMostFreq(vocabList,fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    sortedFreq = sorted(freqDict.items(),key = operator.itemgetter(1),reverse = True)
    
    return sortedFreq[:30]

def localWords(feed1,feed0):
    import feedparser
    docList = []; classList = [];fullText = []
    minLen = min(len(feed1['entries']),len(feed0['entries']))
#     print("-----------minLen",minLen)
    
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    top30Words = calcMostFreq(vocabList,fullText)
    for pairW in top30Words:
        if pairW[0] in vocabList:vocabList.remove(pairW[0])
            
   
    trainingSet = list(range(2*minLen));testSet = []
   
 
    for i in range(20):
#         print("-----i-----",i)
        randIndex = int(random.uniform(0,len(trainingSet)))
        randIndex = int(random.uniform(0,len(trainingSet)))
#         print("-----randIndex",randIndex,"------trainingSet",trainingSet)
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
        
    trainMat = [];trainClasses = []
    
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList,docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam)!=classList[docIndex]:
            errorCount+=1
    print('************the error rate is :************** ',float(errorCount)/len(testSet))
    return vocabList,p0V,p1V

上述代码类似程序清单4-5中的函数spamTest(),不过添加了新的功能，代码中引入了一个辅助函数calcMostFreq(), 该函数遍历词汇表中的每一个词并统计它在文本中出现的次数，然后根据出现次数从高到低对词典进行排序，最后返回排序最高的30个单词。你很快就会明白这个函数的重要性。

下一个函数localWords()使用两个RSS源作为参数。RSS源在函数外导入，这样做的原因是RSS源会随时间而改变。如果想通过改变代码来比较程序执行的差异，就应该使用相同的输入。重新加载RSS源就会得到新的数据，但很难确定是代码愿意还是输入原因导致输出结果的改变。函数local Words()与程序清单4-5中的spamTest（）函数几乎相同，区别在于这里访问的是RSS源而不是文件。然后调用函数calcMostFreq()来获得排序最高的30个单词并随后将它们移除3.函数的剩余部分与spamTest() 基本类似，不同的是最后一行要返回下面要用到的值。

词汇白中的一小部分单词却占用了所有文本用词的一大部分，这种现象的原因是因为语言中大部分都是冗余和结构辅助性内容。另一个常用的方法是不仅移除高频词，同时从某个预定词表中移除结构上的辅助词。该词表称为停用词表（stop word list）。

In [88]:
import feedparser
ny = feedparser.parse('https://blog.csdn.net/qq_34243930/rss/list') #界面新闻RSS地址
sf = feedparser.parse('http://news.stockstar.com/rss/xml.aspx?file=xml/stock/2.xml') #证券之星





In [89]:
len(sf['entries'])

30

In [90]:
len(ny['entries'])

20

In [91]:

vocabList,pSF,pNY = localWords(ny,sf)

************the error rate is :**************  0.4


In [92]:
vocabList

['一般用vscode写',
 '保险核心业务处理系统',
 '15亿元',
 '转为列表',
 '研究建立投资者保护专项补偿基金',
 'center',
 '在引导价值投资理念',
 '目前伯克希尔对富国银行的持仓已经从二季度末的2',
 '由于coco数据格式分为三部分',
 '发布了招股说明书申报稿',
 '这种突破是否具有持续性',
 '第11',
 '在此来总结一下',
 'are',
 '函数详解',
 '11亿元',
 '迈瑞医疗成为近一周调研机构数量最多的股票之一',
 'have',
 '除了上述事项',
 '云南白药',
 '9月3日的18',
 'doesn',
 '蔬菜价格涨幅较大',
 '这周末',
 'whale',
 '就是从matlab读出来的',
 '售后支持服务等多方面的优势',
 '约合170亿元人民币',
 'future',
 '曾多次引入财务投资者',
 '如何改变过去扭曲的投研现象',
 '公司的上述行为违反了本所',
 '华海药业',
 '有的表示这是在复制tcl李东升的操作',
 'error',
 'bashrc文件介绍',
 '我们需要更多能吸收风险的股权投资来培育和支持创新企业',
 '要自定义运行和调试创建launch',
 '人民银行需要建立和完善现代中央银行制度',
 '市场传出美股科技股近几个月连续疯涨的背后',
 '外资持有我国流通股市值比重达4',
 '疫情中',
 '当assert',
 '合计解禁市值663',
 'ppi',
 '的转换',
 '我们对此感到震惊和不解',
 '媒报道美国政府考虑将中芯国际列入贸易黑名单的消息一石激起千层浪',
 '解禁股主要为发行前股份限售流通',
 '2fss2020090600000027',
 'type',
 '此次解禁市值规模为161',
 '在市场突发剧烈调整之际',
 '语句失败的时候',
 '安装vscode',
 '下定决心',
 '解禁市值高达194',
 '普通股持股比例从5',
 '中芯国际对此发表严正声明',
 '持股5',
 '这里我的主目录是',
 '26万股',
 '二季度',
 'korn',
 '金域医学发布公告称',
 '在列表末尾添加新的对象',
 '纳斯达克巨鲸',
 '伯克希尔近日已卖出1亿股富国银行股票',
 '

In [93]:
pSF

array([-6.07073773, -5.37759055, -5.37759055, ..., -6.07073773,
       -6.07073773, -6.07073773])

In [94]:
pNY

array([-5.48893773, -5.48893773, -5.48893773, ..., -5.48893773,
       -5.48893773, -5.48893773])

为了的到错误率的精确估计，应该多次进行上述实验，然后取平均值。这里的错误率要远远高于垃圾邮件中的错误率。由于这里关注的是单词概率而不是实际分类，因此这个问题倒不严重。可以通过函数calcMostFreq()改变要移除的单词数目，然后观察错误率的变化情况。

#### 4.7.2 分析数据：显示地域相关的用词

可以先对向量pSF与pNY进行排序，然后按照顺序将词打印出来。下面的最后一段代码会完成这部分工作。



$$程序清单4-7 最具表征性的词汇显示函数$$

In [95]:
def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V = localWords(ny,sf)
    topNY = [];topSF = []
    for i in range(len(p0V)):
#         print('-----p0V[i]-----',p0V[i])
        if p0V[i]>-6.0: topSF.append((vocabList[i],p0V[i]))
        if p1V[i]>-6.0: topNY.append((vocabList[i],p1V[i]))
#     print('-----topSF---',topSF)
#     print('----topNY----',topNY)
    sortedSF =sorted(topSF, key = lambda pair: pair[1],reverse= True)
#     print("----sortedSF---",sortedSF)
    
    print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print(item[0])
    sortedNY = sorted(topNY, key = lambda pair:pair[1],reverse= True)
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print(item[0])
    

程序清单4-7中的函数getTopWords()使用两个RSS源作为输入，然后训练并测试朴素贝叶斯分类器，返回使用的概率值。然后创建两个列表用于元组的储存。与之前返回排名最高的X个单词不同，这里可以返回大于某个阈值的所有词。这些元组会按照它们的条件概率进行排序。

下面看一下实际的运行效果

In [96]:
getTopWords(ny,sf)

************the error rate is :**************  0.4
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
center
img
align
ssimg
title
sscms
src
jpg
红周刊
今年以来
62亿元
较当天收盘价折价4
amp
目前伯克希尔对富国银行的持仓已经从二季度末的2
发布了招股说明书申报稿
11亿元
9月3日的18
这周末
whale
约合170亿元人民币
曾多次引入财务投资者
市场传出美股科技股近几个月连续疯涨的背后
我们对此感到震惊和不解
媒报道美国政府考虑将中芯国际列入贸易黑名单的消息一石激起千层浪
在市场突发剧烈调整之际
下定决心
普通股持股比例从5
中芯国际对此发表严正声明
26万股
二季度
纳斯达克巨鲸
伯克希尔近日已卖出1亿股富国银行股票
全周来看
其余部分均用于补充流动资金
据13f报告
提前明确前三个季度业绩预喜的公司合计175家
2fss2020090700000237
的报道均为不实新闻
是何意味
53元
在今年二季度
分析该公司招股书
单日一度暴跌5
501份三季报预告仅三成公司业绩预喜
卖出股份数分别为2000万股
将不再披露对其直接持股信息
由于欧美新冠疫情的爆发
降低至3
从六方面加快推动电子商务高质量发展
美的遭董事长方洪波和神秘资金抛售
最终民用厂商
五是开展惠民惠企行动
近期美股持续出现大幅震荡
从披露公司占各市场比重看
2fig2020090600000055
商务部副部长钱克明9月5日在2020中国电子商务大会上表示
软银被曝购入40亿美元科技股
全周合计流出222
公司有可能对自己的采销和营收数据进行了一定的调整
占比12
2017
三孚新科
且与多个美国及国际知名的半导体设备供货商
而富国银行却下跌2
最新结果是
纳指开始连续两天下跌
从深交所已经披露的9月2日
但就目前来看
美国大选进入胶着状态
8月期间
胡振明
媒纷纷称其为
中芯国际愿以诚恳
在上周升至12000点的历史新高后
揭开帷幕
日本软银集团
早年曾在新三板挂牌的三孚新科选择在沪市科创板上市
仍有501家公司提前公布了三季报业绩预期情况
目前两市已有501家公司发布了三季报业绩预告
道琼斯指数

最后输出的单词很有意思。值得注意的现象是程序输出了大量的停用词。移除固定的停用词看看结果会如何变化也十分有趣。以我的经验来看没这样做的话，分类错误率也会降低。

### 4.8 本章小节

对于分类而言，使用概率又是要比使用硬规则更有效。贝叶斯概率及贝叶斯准则提供了一种利用已知值来估计未知概率的有效方法。

可以通过特征之间的条件独立性假设，降低对数据量的需求。独立性假设是指一个词的出现概率并不依赖于文档中的其他词。当然我们也知道这个假设过于简单。这就是之所以称之为朴素贝叶斯的原因。尽管条件独立性假设并不正确，但是朴素贝叶斯仍然是一种有效的分类器。

利用现代编程语言来实现朴素贝叶斯需要考虑很多实际因素。下溢出就是其中一个问题，它可以通过对概率取对数来解决。词袋模型在解决文档分类问题上比词集模型有所提高。还有其他一些方面的改进，比如说移除停用词，当然也可以花大量时间对切分器进行优化。

本章学习到的概率理论将在后续章节中用到，另外本章也给出了有关贝叶斯概率理论全面具体的介绍。接下来的一章将暂时不再讨论概率理论这一话题，介绍另一种称作Logistic回归的分类方法集一些优化算法。